In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Load the IMDB dataset
max_features = 20000  # Number of words to consider as features
maxlen = 100  # Cut texts after this number of words (for padding)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(f'{len(x_train)} train sequences')
print(f'{len(x_test)} test sequences')

print('Pad sequences (samples x time)')
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')

Loading data...
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 100)
x_test shape: (25000, 100)


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 128, input_length=maxlen))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
print('Training model...')
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    validation_data=(x_test, y_test))

Training model...
Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 23ms/step - accuracy: 0.7267 - loss: 0.5268 - val_accuracy: 0.8353 - val_loss: 0.3819
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 34s 19ms/step - accuracy: 0.9022 - loss: 0.2567 - val_accuracy: 0.8396 - val_loss: 0.3841
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9415 - loss: 0.1623 - val_accuracy: 0.8344 - val_loss: 0.4278
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9638 - loss: 0.1017 - val_accuracy: 0.8339 - val_loss: 0.5031
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9806 - loss: 0.0626 - val_accuracy: 0.8246 - val_loss: 0.7258
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.9836 - loss: 0.0486 - val_accuracy: 0.8233 - val_loss: 0.6253
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.9916 - loss: 0.0275 - val_accuracy: 0.8247 - val_loss: 0.8229
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9933

In [ ]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print(f'Test score: {score}')
print(f'Test accuracy: {acc}')

782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8206 - loss: 0.8078
Test score: 0.7971096038818359
Test accuracy: 0.8228399753570557


In [ ]:
# Decode and display review text along with its predicted sentiment
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

def decode_review(encoded_review):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in encoded_review])

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Select a sample review for prediction
sample_review = x_test[2000]
decoded_review = decode_review(sample_review)

In [ ]:
# Predict sentiment using the trained LSTM model
prediction = model.predict(np.expand_dims(sample_review, axis=0))
sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

print("\nSample Review:")
print(f"Review: {decoded_review}")
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Sample Review:
Review: this one brilliantly you can't help but wonder if he is really out there i reckon he and the other main cast members probably had nightmares for weeks after doing this movie as it's so intense when i first saw it i was just ? channels on the remote late one evening i got hooked within minutes look up www answers com for ? ? who is the character that carlos the is based on for both i remember reading about ? arrest in the paper in 1997 it was front page for weeks through the trial after his arrest
Predicted Sentiment: Positive


In [ ]:
# Select a sample review for prediction
sample_review = x_test[100]
decoded_review = decode_review(sample_review)

In [ ]:
# Predict sentiment using the trained LSTM model
prediction = model.predict(np.expand_dims(sample_review, axis=0))
sentiment = "Positive" if prediction[0][0] > 0.5 else "Negative"

print("\nSample Review:")
print(f"Review: {decoded_review}")
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

Sample Review:
Review: even lift an eyebrow about this movie was the fact the med teacher was ? in terminator 2 also a movie that was light years ahead of this motion picture massacre anyone who was involved in this movie should be ashamed of themselves for wasting 90 minutes of countless people's time it's no wonder no actor from this movie ever had a ? career in summary this movie is so bad i feel dirty and need a shower worst movie in history gigli was better prom night the remake was better and dare i say it saw iv was better
Predicted Sentiment: Negative
